# Model
This notebook is for modeling, we use different base models and use stacking to ensemble them.

## Packages Load

In [1]:
import pandas as pd
import numpy as np
import multiprocessing
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb
import xgboost as xgb
import catboost as cab
import time
from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import numpy as np
import time
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook

In [2]:
# Read Files
events = pd.read_pickle('data/events.pkl')
attr = pd.read_pickle('data/attr.pkl')
session = pd.read_pickle('data/session.pkl')
submission = pd.read_csv('data/submission.csv')

## Load Data

In [82]:
training = pd.read_pickle('training_att.pkl')
prediction = pd.read_pickle('prediction_att.pkl')

In [83]:
training.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days,event_count,purchase_count_1_week,purchase_count_2_week,purchase_count_3_week,purchase_count_4_week,session_count,mean_sessions_duration,...,attr_76_mean,attr_76_max,attr_77_mean,attr_77_max,attr_78_mean,attr_78_max,country,os_name,city,region
0,b5291d394748a326fb4ab798c513a2312b926aeddee46c...,False,False,87,0,0,0,0,87,8.613168e+05,...,0.0,0.0,0.000000,0.0,1.0,1.0,OM,Android OS,muscat,ma
1,c1105124087971191cf1e992a0870eb6b7f507bc236f00...,False,False,37,0,0,0,0,37,0.000000e+00,...,0.0,0.0,0.000000,0.0,1.0,1.0,AU,Android OS,melbourne,vic
2,797da9e3676e77980c7fc938c6bb7c9d70ef826e861f88...,False,False,265,0,0,0,5,265,7.463017e+06,...,0.0,0.0,0.000000,0.0,1.0,1.0,US,iOS,smiths station,al
3,09b6f8dc73bb48e8d9834647f77f74b55e42f1bf39b4f7...,False,False,914,0,0,0,0,914,2.455326e+07,...,0.0,0.0,3.846154,10.0,1.0,1.0,CA,Android OS,brampton,on
4,3f2eae6596d67cee8a6ce19e3d77ef414f5007b72542ce...,False,False,12,0,0,0,0,12,0.000000e+00,...,0.0,0.0,0.000000,0.0,1.0,1.0,IN,Android OS,mumbai,mh


In [84]:
training.columns

Index(['user_id_hash', 'user_purchase_binary_7_days',
       'user_purchase_binary_14_days', 'event_count', 'purchase_count_1_week',
       'purchase_count_2_week', 'purchase_count_3_week',
       'purchase_count_4_week', 'session_count', 'mean_sessions_duration',
       'spend', 'event_gap', 'session_gap', 'purchase_gap', 'life_time',
       'event_45', 'event_1', 'event_5', 'event_6', 'event_14', 'event_4',
       'event_40', 'event_7', 'event_41', 'event_3', 'event_42', 'attr_0_mean',
       'attr_0_max', 'attr_1_mean', 'attr_1_max', 'attr_3_mean', 'attr_3_max',
       'attr_4_mean', 'attr_4_max', 'attr_6_mean', 'attr_6_max', 'attr_8_mean',
       'attr_8_max', 'attr_9_mean', 'attr_9_max', 'attr_11_mean',
       'attr_11_max', 'attr_13_mean', 'attr_13_max', 'attr_14_mean',
       'attr_14_max', 'attr_15_mean', 'attr_15_max', 'attr_16_mean',
       'attr_16_max', 'attr_17_mean', 'attr_17_max', 'attr_18_mean',
       'attr_18_max', 'attr_19_mean', 'attr_19_max', 'attr_20_mean',
      

### Create More Features

In [85]:
def create_event_ratio(df):
    
    event_cols=['event_45', 'event_1', 'event_5', 'event_6', 'event_14',
            'event_4', 'event_40', 'event_7', 'event_41', 'event_3', 'event_42']
    
    for event_col in event_cols:
        
        df[event_col+'_ratio'] = df[event_col]/df['event_count']
    df['purchase_count'] = df['purchase_count_1_week']+df['purchase_count_2_week']+df['purchase_count_3_week']+df['purchase_count_4_week']
    df['purchase_ratio'] = df['purchase_count']/df['event_count']
        
    return df
    

In [86]:
training = create_event_ratio(training)
prediction = create_event_ratio(prediction)

### Categorical Feature Encoding

In [87]:
# One-Hot Encoding
def oh_encoding(df, col='os_name', na_value='missing'):
#     df[col] = df[col].fillna(value=na_value)
#     dummy = pd.get_dummies(df[col]).drop(columns = [na_value])
    dummy = pd.get_dummies(df[col])
    dummy['iOS'], dummy['os_android'] = dummy['iOS']+dummy['iPhone OS'], dummy['Android OS']
    dummy = dummy.loc[:,['iOS']]
    df = pd.concat((df, dummy), axis=1)
    return df

In [88]:
training = oh_encoding(training)
prediction = oh_encoding(prediction)

In [89]:
small_country = set(training.country.value_counts()[30:].keys())
small_region = set(training.region.value_counts()[50:].keys())
small_city = set(training.city.value_counts()[50:].keys())

def other_country(df):
    df['country'] = df['country'].apply(lambda x: 'other' if x in small_country else x)
    return df

def other_region(df):
    df['region'] = df['region'].apply(lambda x: 'other' if x in small_region else x)
    return df

def other_city(df):
    df['city'] = df['city'].apply(lambda x: 'other' if x in small_city else x)
    return df

In [90]:
training = other_country(training)
training = other_region(training)
training = other_city(training)

prediction = other_country(prediction)
prediction = other_region(prediction)
prediction = other_city(prediction)

In [91]:
train_y2 = training.user_purchase_binary_14_days
# Mean Encoding Country
def reg_target_encoding(train, col = "country", splits=5):

    train[f'{col}_mean_enc'] = 0
#     train_y2 = training.user_purchase_binary_14_days.values
    skf = StratifiedKFold(splits, shuffle=True, random_state=111)
    
    for computing_index, encoding_index in skf.split(training,train_y2):
        computing, encoding = train.iloc[computing_index], train.iloc[encoding_index]
        means = computing.groupby(col).user_purchase_binary_14_days.mean()
        train.loc[encoding_index,f'{col}_mean_enc'] = encoding[col].map(means)
    return train

In [92]:
training =reg_target_encoding(training, col = "country")
training =reg_target_encoding(training, col = "region")
training =reg_target_encoding(training, col = "city")



In [93]:
def mean_encoding_test(test, train, col = "country"):

    global_mean = train.user_purchase_binary_14_days.mean()
    mean_device_type = train.groupby(col).user_purchase_binary_14_days.mean()
    test[f'{col}_mean_enc'] = test[col].map(mean_device_type)
    test[f'{col}_mean_enc'].fillna(global_mean, inplace=True)
    
    return test
    

def feature_encoding(df, training_df):
#     # One-Hot Encoding OS
#     df['os_name'] = df['os_name'].fillna(value='Missing')
#     os = pd.get_dummies(df['os_name']).drop(columns = ['Missing'])
#     os['os_ios'], os['os_android'] = os['iOS']+os['iPhone OS'], os['Android OS']
#     os = os.loc[:,['os_ios', 'os_android']]
#     df = pd.concat((df,os), axis=1)
    
    # Mean Encoding country
    df = mean_encoding_test(df, training_df, col = "country")   
    # Mean Encoding region   
    df = mean_encoding_test(df, training_df, col = "region")    
    # Mean Encoding region   
    df = mean_encoding_test(df, training_df, col = "city")
    return df

In [94]:
prediction = feature_encoding(prediction, training)

### Numeric Feature Scale

In [95]:
training.describe()

,event_count,purchase_count_1_week,purchase_count_2_week,purchase_count_3_week,purchase_count_4_week,session_count,mean_sessions_duration,spend,event_gap,session_gap,...,event_7_ratio,event_41_ratio,event_3_ratio,event_42_ratio,purchase_count,purchase_ratio,iOS,country_mean_enc,region_mean_enc,city_mean_enc
count,619423.000000,619423.000000,619423.000000,619423.000000,619423.000000,619423.000000,6.187780e+05,619423.0,6.191750e+05,6.187780e+05,...,619175.000000,619175.000000,619175.000000,619175.000000,619423.000000,619175.000000,619423.000000,618883.000000,619149.000000,619151.000000
mean,170.134648,0.030569,0.048732,0.049472,0.279554,170.134648,3.821014e+06,NaN,2.606123e+09,2.876872e+09,...,0.025014,0.005987,0.005498,0.004464,0.408327,0.000679,0.335543,0.009898,0.009894,0.009895
std,484.164967,0.584570,0.742280,0.763450,2.555597,484.164967,1.281479e+07,NaN,1.571356e+09,1.514580e+09,...,0.028255,0.010942,0.010289,0.008850,3.271250,0.004899,0.472180,0.005838,0.004883,0.003638
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,6.048002e+08,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,14.000000,0.000000,0.000000,0.000000,0.000000,14.000000,0.000000e+00,0.0,1.266770e+09,1.529740e+09,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003808,0.006046,0.009685
50%,37.000000,0.000000,0.000000,0.000000,0.000000,37.000000,4.491725e+04,27.0,2.480097e+09,2.796741e+09,...,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012712,0.011414,0.009737
75%,113.000000,0.000000,0.000000,0.000000,0.000000,113.000000,2.262149e+06,63.0,3.883938e+09,4.140815e+09,...,0.041096,0.008929,0.007402,0.002439,0.000000,0.000000,1.000000,0.015214,0.013981,0.009771
max,24539.000000,65.000000,88.000000,102.000000,444.000000,24539.000000,8.111539e+08,inf,5.849823e+09,5.849990e+09,...,0.333333,0.166667,0.142857,0.100000,445.000000,0.500000,1.000000,0.015350,0.022561,0.025332


Turn milliseconds gap into hour gap

In [96]:
attr_columns = [col for col in training.columns if 'attr' in col]

In [97]:
# Turn gap into hours, and duration into minutes, life_time into days
def tranform_gap(df):
    gap_cols = [col for col in df.columns if 'gap' in col]
    df[gap_cols] = df[gap_cols]/3600000
    df['mean_sessions_duration'] = df['mean_sessions_duration']/60000
    df['life_time'] = df['life_time']/3600000/24
    return df

In [98]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [99]:
def fill_missing(df):
    attr_columns = [col for col in df.columns if 'attr' in col]
    values = {'country': 'other', 
              'os_name': 'other',
              'city': 'other',
              'region': 'other',
              'mean_sessions_duration': 0,
              'event_gap': df['event_gap'].mean(),              
              'session_gap': df['event_gap'].mean(),
              'life_time': df['life_time'].mean(),
              'purchase_gap':  df['purchase_gap'].max()*2,
              'event_45_ratio': df['event_45_ratio'].mean(),
              'event_1_ratio': df['event_1_ratio'].mean(),
              'event_5_ratio': df['event_5_ratio'].mean(),
              'event_6_ratio': df['event_6_ratio'].mean(),
              'event_14_ratio': df['event_14_ratio'].mean(),
              'event_4_ratio': df['event_4_ratio'].mean(),
              'event_40_ratio': df['event_40_ratio'].mean(),
              'event_7_ratio': df['event_7_ratio'].mean(),
              'event_41_ratio': df['event_41_ratio'].mean(),
              'event_3_ratio': df['event_3_ratio'].mean(),
              'event_42_ratio': df['event_42_ratio'].mean(), 
              'purchase_ratio': df['purchase_ratio'].mean(),
              'attr_67_max': df['attr_67_max'].mean(),
              'country_mean_enc': df['country_mean_enc'].mean(),
              'region_mean_enc': df['region_mean_enc'].mean(),
              'city_mean_enc': df['city_mean_enc'].mean(),
              'attr_0_mean': df['attr_0_mean'].mean(),
              'attr_0_max': df['attr_0_max'].mean(),
            'attr_1_mean': df['attr_1_mean'].mean(),
            'attr_1_max': df['attr_1_max'].mean(),
            'attr_3_mean': df['attr_3_mean'].mean(),
            'attr_3_max': df['attr_3_max'].mean(),
            'attr_4_mean': df['attr_4_mean'].mean(),
            'attr_4_max': df['attr_4_max'].mean(),
            'attr_6_mean': df['attr_6_mean'].mean(),
            'attr_6_max': df['attr_6_max'].mean(),
            'attr_8_mean': df['attr_8_mean'].mean(),
            'attr_8_max': df['attr_8_max'].mean(),
            'attr_9_mean': df['attr_9_mean'].mean(),
            'attr_9_max': df['attr_9_max'].mean(),
            'attr_11_mean': df['attr_11_mean'].mean(),
            'attr_11_max': df['attr_11_max'].mean(),
            'attr_13_mean': df['attr_13_mean'].mean(),
            'attr_13_max': df['attr_13_max'].mean(),
            'attr_14_mean': df['attr_14_mean'].mean(),
            'attr_14_max': df['attr_14_max'].mean(),
            'attr_15_mean': df['attr_15_mean'].mean(),
            'attr_15_max': df['attr_15_max'].mean(),
            'attr_16_mean': df['attr_16_mean'].mean(),
            'attr_16_max': df['attr_16_max'].mean(),
            'attr_17_mean': df['attr_17_mean'].mean(),
            'attr_17_max': df['attr_17_max'].mean(),
            'attr_18_mean': df['attr_18_mean'].mean(),
            'attr_18_max': df['attr_18_max'].mean(),
            'attr_19_mean': df['attr_19_mean'].mean(),
            'attr_19_max': df['attr_19_max'].mean(),
            'attr_20_mean': df['attr_20_mean'].mean(),
            'attr_20_max': df['attr_20_max'].mean(),
            'attr_21_mean': df['attr_21_mean'].mean(),
            'attr_21_max': df['attr_21_max'].mean(),
            'attr_67_mean': df['attr_67_mean'].mean(),
            'attr_67_max': df['attr_67_max'].mean(),
            'attr_75_mean': df['attr_75_mean'].mean(),
            'attr_75_max': df['attr_75_max'].mean(),
            'attr_76_mean': df['attr_76_mean'].mean(),
             'attr_76_max': df['attr_76_max'].mean(),
             'attr_77_mean': df['attr_77_mean'].mean(),
             'attr_77_max': df['attr_77_max'].mean(),
             'attr_78_mean': df['attr_78_mean'].mean(),
             'attr_78_max': df['attr_78_max'].mean()}
    df = df.fillna(value=values, inplace=False)
    
    return df

In [100]:
numeric_feature = training.select_dtypes(include=[np.number]).columns.tolist()

In [103]:
prediction.spend.max()

22190.0

In [102]:
prediction.loc[prediction.spend==np.inf, 'spend']=999

In [104]:
def feature_processing(df):
    scaler = StandardScaler()
    # Winsorization
    for col in numeric_feature:
        df[col] = np.clip(df[col], 0, np.quantile(df[col],0.98))
    
    df['event_count'] = np.log(df['event_count']+1)
    df[numeric_feature] = scaler.fit_transform(df[numeric_feature])
    df[numeric_feature] = fill_missing(df[numeric_feature])
    
    return df

In [105]:
training = feature_processing(training)

/home/wtq920828/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3652: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/home/wtq920828/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float16, float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/wtq920828/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, float16, float32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [106]:
prediction = feature_processing(prediction)

/home/wtq920828/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3652: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/home/wtq920828/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float16, float32, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/wtq920828/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, float16, float32, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [107]:
training.columns

Index(['user_id_hash', 'user_purchase_binary_7_days',
       'user_purchase_binary_14_days', 'event_count', 'purchase_count_1_week',
       'purchase_count_2_week', 'purchase_count_3_week',
       'purchase_count_4_week', 'session_count', 'mean_sessions_duration',
       'spend', 'event_gap', 'session_gap', 'purchase_gap', 'life_time',
       'event_45', 'event_1', 'event_5', 'event_6', 'event_14', 'event_4',
       'event_40', 'event_7', 'event_41', 'event_3', 'event_42', 'attr_0_mean',
       'attr_0_max', 'attr_1_mean', 'attr_1_max', 'attr_3_mean', 'attr_3_max',
       'attr_4_mean', 'attr_4_max', 'attr_6_mean', 'attr_6_max', 'attr_8_mean',
       'attr_8_max', 'attr_9_mean', 'attr_9_max', 'attr_11_mean',
       'attr_11_max', 'attr_13_mean', 'attr_13_max', 'attr_14_mean',
       'attr_14_max', 'attr_15_mean', 'attr_15_max', 'attr_16_mean',
       'attr_16_max', 'attr_17_mean', 'attr_17_max', 'attr_18_mean',
       'attr_18_max', 'attr_19_mean', 'attr_19_max', 'attr_20_mean',
      

In [108]:
prediction.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days,event_count,purchase_count_1_week,purchase_count_2_week,purchase_count_3_week,purchase_count_4_week,session_count,mean_sessions_duration,...,event_7_ratio,event_41_ratio,event_3_ratio,event_42_ratio,purchase_count,purchase_ratio,iOS,country_mean_enc,region_mean_enc,city_mean_enc
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.01,0.02,-0.330242,-0.051871,-0.061426,-0.071893,-0.121927,-0.289638,-0.266923,...,0.484329,-0.552368,-0.539847,-0.510521,-0.12907,-0.143336,1.404762,0.910602,0.771495,-0.03688
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.01,0.02,0.073087,-0.051871,-0.061426,-0.071893,-0.121927,-0.247201,-0.288907,...,-0.172361,-0.552368,-0.539847,-0.510521,-0.12907,-0.143336,-0.711865,-1.234977,-0.794769,-0.03688
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.01,0.02,-0.222496,-0.051871,-0.061426,-0.071893,-0.121927,-0.280797,-0.264945,...,1.411110,-0.552368,-0.539847,-0.510521,-0.12907,-0.143336,-0.711865,-1.234977,-0.794769,-0.03688
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.01,0.02,0.964557,-0.051871,-0.061426,-0.071893,-0.121927,0.030411,0.278374,...,-0.196419,1.657191,1.341695,0.580075,-0.12907,-0.143336,-0.711865,-1.038442,-0.794769,-0.03688
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.01,0.02,-1.284093,-0.051871,-0.061426,-0.071893,-0.121927,-0.326771,-0.288907,...,-0.883776,-0.552368,-0.539847,-0.510521,-0.12907,-0.143336,-0.711865,-1.038442,-0.890371,-0.03688


## Modeling

In [109]:
X = training.drop(columns =['user_id_hash', 'user_purchase_binary_7_days', 'user_purchase_binary_14_days','country','os_name','city','region','iOS']).values
y1, y2 = training['user_purchase_binary_7_days'].values, training['user_purchase_binary_14_days'].values
X_test = prediction.drop(columns =['user_id_hash', 'user_purchase_binary_7_days', 'user_purchase_binary_14_days','country','os_name','city','region','iOS']).values

In [110]:
y1 = y1.astype(int)
y2 = y2.astype(int)

In [111]:
y1.min()

0

### First Layer Model
On the first layer, we will use 3 tree boosting models, 2 neural networks, 2 linear models, 1 naive bayes model, 1 knn model.

In [112]:
# for the number of neurons in the hidden unit
def get_model(hidden_size = [1024, 16], dropout=0.1):
    modules = []
    modules.append(nn.Linear(X.shape[1], hidden_size[0]))
    modules.append(nn.ReLU())
    if len(hidden_size)==3:
        modules.append(nn.Linear(hidden_size[0], hidden_size[1]))
        modules.append(nn.ReLU())
        modules.append(nn.Linear(hidden_size[1], hidden_size[-1]))
        modules.append(nn.ReLU())
    else:
        modules.append(nn.Linear(hidden_size[0], hidden_size[-1]))
        modules.append(nn.ReLU())
#     for i, hz in enumerate(hidden_size[1:-1]):
#         modules.append(nn.Linear(hidden_size[i+1], hidden_size[i+2]))
#         modules.append(nn.ReLU())
    if dropout > 0:
        modules.append(nn.Dropout(dropout))
    modules.append(nn.Linear(hidden_size[-1], 1))
    return nn.Sequential(*modules)#.cuda()


In [114]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [115]:
splits = list(StratifiedKFold(n_splits=4, shuffle=True, random_state=42).split(X, y1, y2))

In [116]:
import time

In [117]:
X_test.shape

(312568, 83)

In [118]:
# torch.tensor(X[train_idx], dtype=torch.float32).cuda()
# valid = torch.utils.data.TensorDataset(x_val, y_val)
# valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

X_test_cuda = torch.tensor(X_test, dtype=torch.float32)#.cuda()
test = torch.utils.data.TensorDataset(X_test_cuda)
prediction_batch_size = 4096*4
test_loader = torch.utils.data.DataLoader(test, batch_size=prediction_batch_size, shuffle=False)

In [119]:
from sklearn.metrics import roc_auc_score

In [123]:
def train_model(model, x_train, y_train, x_val, y_val, epochs=10, validate=True):
    
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001) #Optimizer Setting
    
    
    train = torch.utils.data.TensorDataset(x_train, y_train)  #Tranform data to tensor dataset
    valid = torch.utils.data.TensorDataset(x_val, y_val)
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True) #Create data loader
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
    
    loss_fn = torch.nn.BCEWithLogitsLoss(reduction='mean',pos_weight = torch.tensor(10))#.cuda() #Loss function setting
    
    for epoch in range(n_epochs):  #Epoch Iteration
#         scheduler.step()
        start_time = time.time()
        model.train() #Set model to train mode
        avg_loss = 0.
        
        for x_batch, y_batch in tqdm_notebook(train_loader):
#             scheduler.step()
            y_pred = model(x_batch)
#             print(y_pred)
            # scheduler.batch_step()      
            loss = loss_fn(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
        
        avg_loss = avg_loss / len(train_loader)
        
        model.eval()
        
        valid_preds = np.zeros((x_val.size(0)))
        
        if validate:
            avg_val_loss = 0.
            for i, (x_batch, y_batch) in enumerate(valid_loader):
#                 print(type(x_batch))
                y_pred = model(x_batch).detach()
#                 print(y_pred)

                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
                valid_preds[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
            print(y_val.max())
            print(valid_preds.max())
            val_auc =  roc_auc_score(y_val.cpu(), valid_preds)
            elapsed_time = time.time() - start_time
            print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t val_auc={:.4f} \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, val_auc, elapsed_time))
        else:
            elapsed_time = time.time() - start_time
            print('Epoch {}/{} \t loss={:.4f} \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, elapsed_time))
   
    valid_preds = np.zeros((x_val_fold.size(0)))
    
    avg_val_loss = 0.
    for i, (x_batch,  y_batch) in enumerate(valid_loader):
        y_pred = model(x_batch).detach()

        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        valid_preds[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]

    print('Validation loss: ', avg_val_loss)

    test_preds = np.zeros((len(test_loader.dataset)))
    
    prediction_batch_size = 4096*4
    for i, (x_batch) in enumerate(test_loader):
#         print(type(x_batch))
        y_pred = model(x_batch[0]).detach()

        test_preds[i * prediction_batch_size:(i+1) * prediction_batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]

    
    # test_preds_local = np.zeros((len(test_local_loader.dataset)))
    
#     for i, (x_batch,) in enumerate(test_local_loader):
#         y_pred = model(x_batch).detach()

#         test_preds_local[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]

    return valid_preds, test_preds#, test_preds_local

In [124]:
model

Sequential(
  (0): Linear(in_features=83, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=32, bias=True)
  (5): ReLU()
  (6): Dropout(p=0.1)
  (7): Linear(in_features=32, out_features=1, bias=True)
)

In [128]:
train_preds = np.zeros(len(X))
test_preds = np.zeros((len(X_test), len(splits)))
# final_preds = np.zeros((len(test_X), len(splits)))
# final_train = np.zeros((len(train_X), len(splits)))
# test_preds_local = np.zeros((n_test, len(splits)))
n_epochs = 10
batch_size = 32

from tqdm import tqdm
for i, (train_idx, valid_idx) in enumerate(splits):    
    x_train_fold = torch.tensor(X[train_idx], dtype=torch.float32)#.cuda()
    y_train_fold = torch.tensor(y1[train_idx, np.newaxis], dtype=torch.float32)#.cuda()
    x_val_fold = torch.tensor(X[valid_idx], dtype=torch.float32)#.cuda()
    y_val_fold = torch.tensor(y1[valid_idx, np.newaxis], dtype=torch.float32)#.cuda()
    print(f'Fold {i + 1}')
    
    model = get_model(hidden_size=[256,64])
    
    def init_normal(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform(m.weight)

# use the modules apply function to recursively apply the initialization
    model.apply(init_normal)
#     model.cuda()

    valid_preds_fold, test_preds_fold = train_model(model,
                                                    x_train_fold,                                                    
                                                    y_train_fold, 
                                                    x_val_fold, 
                                                    y_val_fold, validate=True)

    train_preds[valid_idx] = valid_preds_fold
    test_preds[:, i] = test_preds_fold
# final_train[:,0] = train_preds
# final_preds[:,0] = test_preds.mean(1)

Fold 1


/home/wtq920828/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


tensor(1.)
0.9958223104476929
Epoch 1/10 	 loss=0.0519 	 val_loss=0.0435 	 val_auc=0.9954 	 time=39.01s


tensor(1.)
0.9896971583366394
Epoch 2/10 	 loss=0.0441 	 val_loss=0.0425 	 val_auc=0.9962 	 time=41.38s


tensor(1.)
0.9991831183433533
Epoch 3/10 	 loss=0.0397 	 val_loss=0.0377 	 val_auc=0.9964 	 time=42.60s


...

tensor(1.)
0.9995179176330566
Epoch 6/10 	 loss=0.0386 	 val_loss=0.0352 	 val_auc=0.9965 	 time=45.61s


tensor(1.)
0.9978925585746765
Epoch 7/10 	 loss=0.0374 	 val_loss=0.0401 	 val_auc=0.9963 	 time=51.13s


tensor(1.)
0.9951339364051819
Epoch 8/10 	 loss=0.0385 	 val_loss=0.0381 	 val_auc=0.9963 	 time=54.31s


tensor(1.)
0.9991773962974548
Epoch 9/10 	 loss=0.0391 	 val_loss=0.0424 	 val_auc=0.9963 	 time=56.13s


tensor(1.)
0.9998010993003845
Epoch 10/10 	 loss=0.0382 	 val_loss=0.0357 	 val_auc=0.9966 	 time=56.14s
Validation loss:  0.03574693559708398


In [129]:
train_preds.shape

(619423,)

In [130]:
test_preds.shape

(312568, 4)

In [131]:
nn_validation_1 = train_preds
nn_prediction_1 = np.mean(test_preds, axis=1)

In [133]:
np.save('./prediction/nn_validation_1', nn_validation_1)
np.save('./prediction/nn_prediction_1', nn_prediction_1)

In [135]:
train_preds_2 = np.zeros(len(X))
test_preds_2 = np.zeros((len(X_test), len(splits)))
# final_preds = np.zeros((len(test_X), len(splits)))
# final_train = np.zeros((len(train_X), len(splits)))
# test_preds_local = np.zeros((n_test, len(splits)))
n_epochs = 15
batch_size = 32

from tqdm import tqdm
for i, (train_idx, valid_idx) in enumerate(splits):    
    x_train_fold = torch.tensor(X[train_idx], dtype=torch.float32)#.cuda()
    y_train_fold = torch.tensor(y2[train_idx, np.newaxis], dtype=torch.float32)#.cuda()
    x_val_fold = torch.tensor(X[valid_idx], dtype=torch.float32)#.cuda()
    y_val_fold = torch.tensor(y2[valid_idx, np.newaxis], dtype=torch.float32)#.cuda()
    print(f'Fold {i + 1}')
    
    model = get_model(hidden_size=[256,64])
    
    def init_normal(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform(m.weight)

# use the modules apply function to recursively apply the initialization
    model.apply(init_normal)
#     model.cuda()

    valid_preds_fold_2, test_preds_fold_2 = train_model(model,
                                                    x_train_fold,                                                    
                                                    y_train_fold, 
                                                    x_val_fold, 
                                                    y_val_fold, validate=True)

    train_preds_2[valid_idx] = valid_preds_fold_2
    test_preds_2[:, i] = test_preds_fold_2

Fold 1


/home/wtq920828/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


tensor(1.)
0.9968728423118591
Epoch 1/15 	 loss=0.0566 	 val_loss=0.0445 	 val_auc=0.9976 	 time=38.22s


tensor(1.)
0.9999994039535522
Epoch 2/15 	 loss=0.0421 	 val_loss=0.0436 	 val_auc=0.9982 	 time=40.20s


tensor(1.)
0.9999995231628418
Epoch 3/15 	 loss=0.0381 	 val_loss=0.0325 	 val_auc=0.9985 	 time=38.95s


...

tensor(1.)
1.0
Epoch 11/15 	 loss=0.0343 	 val_loss=0.0418 	 val_auc=0.9981 	 time=57.19s


tensor(1.)
1.0
Epoch 12/15 	 loss=0.0342 	 val_loss=0.0350 	 val_auc=0.9982 	 time=57.25s


tensor(1.)
1.0
Epoch 13/15 	 loss=0.0331 	 val_loss=0.0379 	 val_auc=0.9981 	 time=55.97s


tensor(1.)
1.0
Epoch 14/15 	 loss=0.0338 	 val_loss=0.0441 	 val_auc=0.9977 	 time=58.40s


tensor(1.)
1.0
Epoch 15/15 	 loss=0.0316 	 val_loss=0.0487 	 val_auc=0.9977 	 time=57.95s
Validation loss:  0.048705391198192546


In [ ]:
nn_validation_2 = train_preds_2
nn_prediction_2 = np.mean(test_preds_2, axis=1)

In [ ]:
np.save('./prediction/nn_validation_2', nn_validation_2)
np.save('./prediction/nn_prediction_2', nn_prediction_2)

In [ ]:
knn_validation_label_2, knn_prediction_label_2 = stack_prediction(knn_model, X, y2, X_test, fold=5, random_state=42)

In [ ]:
np.save('knn_validation_label_2', knn_validation_label_2)
np.save('knn_prediction_label_2', knn_prediction_label_2)

In [33]:
X.isna().apply(sum,axis=0)

event_count               0
purchase_count            0
session_count             0
mean_sessions_duration    0
spend                     0
event_gap                 0
session_gap               0
life_time                 0
num_places                0
event_45                  0
event_1                   0
event_5                   0
event_6                   0
event_14                  0
event_4                   0
event_40                  0
event_7                   0
event_41                  0
event_3                   0
event_42                  0
purchase_gap              0
event_45_ratio            0
event_1_ratio             0
event_5_ratio             0
event_6_ratio             0
event_14_ratio            0
event_4_ratio             0
event_40_ratio            0
event_7_ratio             0
event_41_ratio            0
event_3_ratio             0
event_42_ratio            0
purchase_ratio            0
country_mean_enc          0
region_mean_enc           0
city_mean_enc       